<a href="https://colab.research.google.com/github/Vaelastraszz/Stack_OF/blob/master/Unsupervised_Tagging_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pickle

tf_idf = pickle.load(open('/content/gdrive/My Drive/Stack_OF/tf_idf.pkl','rb'))
terms =  pickle.load(open('/content/gdrive/My Drive/Stack_OF/terms.pkl','rb'))

In [0]:
liste_p = pickle.load(open('/content/gdrive/My Drive/Stack_OF/liste_posts.pkl','rb'))

Loading de toutes les instances dont nous aurons besoin pour les traitements afin de créer nos tags de manière non supervisé

<h1> Creating tags algorithm </h1>

<h2> NMF Topics model </h2>

We will use the other results from other notebooks to generate nmf matrix

In [0]:
from sklearn.decomposition import NMF

In [0]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [53]:
no_topics = 35

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tf_idf)

no_top_words = 10
display_topics(nmf, terms, no_top_words)

Topic 0:
sql server query procedure linq client connection express statement management
Topic 1:
table column row query id select record sql mysql key
Topic 2:
file directory folder path read copy config upload script batch
Topic 3:
class public interface name member instance base property constructor type
Topic 4:
page javascript html browser site web css content link ajax
Topic 5:
net asp mvc framework vb web website ajax site webforms
Topic 6:
string character convert regex expression char match number format return
Topic 7:
studio debug express ide microsoft build option management designer debugger
Topic 8:
project build source eclipse folder svn reference repository dependency directory
Topic 9:
service web wcf client soap call server wsdl config site
Topic 10:
java eclipse library jar jvm applet lang sun api equivalent
Topic 11:
function call return javascript parameter argument pointer int jquery pas
Topic 12:
test unit nunit junit framework coverage integration mock tdd suite


Récupération de la matrice NMF qu'on a créer dans notre partie preprocessing 

<h2> Création des fonctions de tagging pertinents </h2>

In [56]:
liste_p[1]

"Percentage width child element in absolutely positioned parent on Internet Explorer 7 I have an absolutely positioned  containing several children, one of which is a relatively positioned . When I use a percentage-based width on the child , it collapses to '0' width on Internet\xa0Explorer\xa07, but not on Firefox or Safari.\nIf I use pixel width, it works. If the parent is relatively positioned, the percentage width on the child works.\n\nIs there something I'm missing here?\nIs there an easy fix for this besides the pixel-based width on the\nchild?\nIs there an area of the CSS specification that covers this?\n\n"

In [0]:
import itertools

def get_tags_index(j):
    
    id_tag = []
        
    for i in np.nonzero(nmf.transform(tf_idf[j].toarray()))[1]:
        id_tag.append(np.intersect1d(np.argsort(nmf.components_[i])[:-10 - 1:-1], np.nonzero(tf_idf[j].toarray())).tolist())
    
    
    return list(set(list(itertools.chain.from_iterable(id_tag))))

In [0]:
def get_tags_name(liste):
    
    tags_name = []
    
    for i in range(len(liste)):
        
        tags_name.append(terms[liste[i]] if terms[liste[i]] != "net" else ".net"  )
    
    return tags_name

Création d'un duo de fonction permettant de suggérer les tags les plus pertinents possibles de la manière suivante.
Dans un premier temps on va récupérer l'ensemble des topics auxquels le post est suceptible d'appartenir, ensuite on va générer une instersection entre le vecteur tf_idf du post en question et avec le vecteur du topics auquel il est suceptible d'appartenir en récupérant uniquement les top 10 mots les plus représentants du topics. Ainsi si un ou plusieurs des tops mots du topic auquel le post est suceptible d'appartenir est présent dans le post on le fera ressortir. On effectue cette action pour tous les topics ayant une probabilité > 0 de représenter le post en question. 
On va dédoublonner et mettre dans une seule liste tous ces tags. 
On appliquera ces deux fonction sur tous les posts pour ainsi générer une liste de tags pour chaque post.

In [0]:
test_list = []

for i in range(0,50):
    test_list.append (get_tags_name(get_tags_index(i)))

Test de la fonction sur les 50 premières instances de la liste de posts de la DB

In [66]:
test_list

[['build',
  'project',
  'vb',
  'type',
  '.net',
  'control',
  'form',
  'application',
  'convert'],
 ['pixel', 'css', 'element'],
 ['year'],
 ['value', 'month', 'time', 'day', 'display'],
 ['page', 'string', 'header', 'server', 'user', 'http', 'web'],
 ['.net'],
 ['public',
  'web',
  'set',
  'string',
  'linq',
  'return',
  'query',
  'select',
  'service'],
 ['mysql', 'data', 'binary', 'store'],
 ['version',
  'value',
  'build',
  'system',
  'number',
  'time',
  'int',
  'return',
  'test'],
 ['mysql', 'trigger', 'table'],
 ['line',
  'name',
  'time',
  'file',
  'api',
  'type',
  'object',
  'int',
  'return',
  'test'],
 ['int', 'length', 'null'],
 ['data',
  'value',
  'database',
  'row',
  'server',
  'table',
  'structure',
  'schema',
  'event',
  '.net',
  'application',
  'sql',
  'function',
  'query',
  'select',
  'trigger'],
 ['console',
  'class',
  'framework',
  'type',
  '.net',
  'control',
  'thread',
  'application'],
 ['interface', 'time', 'event', '

On peut observer que les tags créé pour les posts ont l'air pour la plupart assez pertinents, même si pour certains posts on peut voir des tags qui ont l'air très précis et pas assez généralistes comme pourrait l'être un tag langage ex : python, java, C# etc. Malgré cela ces tags semblent utilisables pour catégoriser des posts, on effectuera par la suite un tagging de manière supervisé, en utilisant les tags déja présents au sein de stack OF 